In [1]:
import os
import math
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer
import time

def fill_variables(filename, file_path, file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, v_capacity, capacities_depots,
        customers_demands, opening_costs_depots, opening_cost_route, zero_or_not):
    
    line = next(file)
    n_customers = int(line.strip())
    print("Número de customers: " + str(n_customers))
    line = next(file)
    n_available_depots = int(line.strip())
    print("Número de depots disponibles: " + str(n_available_depots))
    line = next(file)
    line = next(file)
            
    count = n_available_depots
    #coordenadas depots
    while(line.strip() and count > 0):
        #print(line.split())
        coordinates_depots.append(line.split())    
        line = next(file)
        count = count -1
            
    #print("Coordenadas depots:")
    #print(coordinates_depots)
    if file_path != "Inst2":
        line = next(file)
    if filename == "coordDas150.dat":
        line = next(file)
    #Coordenadas customers
    while(line.strip()):
        coordinates_customers.append(line.split())
        line = next(file)
            
    #print("Coordenadas customers:")
    #print(coordinates_customers)
            
    line = next(file)
            
    #Capacidad de vehículos
    v_capacity = int(line.strip())
    #print("Capacidad vehículos: " + v_capacity)
    line = next(file)
    line = next(file)
            
    #Capacidad de los depots
    while(line.strip()):
        capacities_depots.append(line.split())
                
        line = next(file)
            
    #print("Capacidades depots:")
    #print(capacities_depots)
            
    line = next(file)
            
    while(line.strip()):
        customers_demands.append(line.strip())
                
        line = next(file)
            
    #print("Demandas customers:")
    #print(customers_demands)
            
    line = next(file)
            
    while(line.strip()):
        opening_costs_depots.append(line.split())
                
        line = next(file)
            
    #print("Costos depots:")
    #print(opening_costs_depots)
            
    line = next(file)
            
            
    opening_cost_route = float(line.strip())
                
    line = next(file)
            
    #print("opening cost route:")
    #print(opening_cost_route)
            
    line = next(file)
    zero_or_not = int(line.strip())
            
    #print("zero o uno: " + zero_or_not)
    return [n_customers, n_available_depots, v_capacity, opening_cost_route, zero_or_not]
    
def distance_cost(a1, a2, b1, b2):
    a1 = float(a1)
    a2 = float(a2)
    b1 = float(b1)
    b2 = float(b2)
    x1 = a1-b1
    x2 = a2-b2
    x1 = pow(x1, 2)
    x2 = pow(x2, 2)
    return math.sqrt(x1 + x2)

In [2]:
file_path = "Inst3"
start = time.time()
for filename in sorted(os.listdir(file_path), key=len):
    end = time.time()
    
    print("Tiempo desde el inicio del programa: " + str(end - start))
    print("Siguiente archivo")
    print(filename + "\n")
    f = os.path.join(file_path, filename)
    # checking if it is a file
    if os.path.isfile(f):
        
        #Contenedores de info inicializados en nada
        n_customers = 0
        n_available_depots = 0
        coordinates_depots = []
        coordinates_customers = []
        v_capacity = 0
        capacities_depots = []
        customers_demands = []
        opening_costs_depots = []
        opening_cost_route = 0
        zero_or_not = 1
        
        #Acá se toma la info de los archivos .dat
        with open(f, 'r') as file:
            lista_var = []
            lista_var = fill_variables(filename, file_path, file, n_customers, n_available_depots, coordinates_depots, coordinates_customers, 
                                       v_capacity, capacities_depots, customers_demands, opening_costs_depots, 
                                       opening_cost_route, zero_or_not)
            n_customers = lista_var[0]
            n_available_depots = lista_var[1]
            v_capacity = lista_var[2]
            opening_cost_route = lista_var[3]
            zero_or_not = lista_var[4]
        
        #Empezar operaciones acá
        #distancias_d2d = []  VIAJE ENTRE 2 DEPOTS NO PERMITIDO
        distancias_d2c = []
        distancias_c2c = []
        distancias_c2d = []
        distancias = []
        #Acá se consiguen los c_ij (costos de caminos)
        for i in coordinates_depots:
            '''for j in coordinates_depots:
                if j==i:
                    continue
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2d.append(distance_cost(i[0], i[1], j[0], j[1]))'''
            for j in coordinates_customers:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_d2c.append(distance_cost(i[0], i[1], j[0], j[1]))
        for i in coordinates_customers:
            for j in coordinates_depots:
                distancias.append(distance_cost(i[0], i[1], j[0], j[1]))
                distancias_c2d.append(distance_cost(i[0], i[1], j[0], j[1]))
                

        for i in range(len(coordinates_customers)):
            for j in range(len(coordinates_customers)):
                b = coordinates_customers[i]
                c = coordinates_customers[j]
                if j==i:
                    continue
                distancias.append(distance_cost(b[0], b[1], c[0], c[1]))
                distancias_c2c.append(distance_cost(b[0], b[1], c[0], c[1]))
                
        #MODELO
        mdl = Model(name="MDRP")
        x = mdl.binary_var_list(len(distancias), name="x")
        
        #print(len(distancias))
        #C_ij * X_ij
        ab = [distancias[i]*x[i] for i in range(len(distancias))]
        mdl.minimize(mdl.sum(ab))

        #X_dj = yd con yd = 1
        for i in range(n_available_depots):
            offset = i*n_customers
            xdj = []
            for j in range(offset, offset+n_customers):
                xdj.append(x[j])
            mdl.add_constraint(sum(xdj) == 1)
            
        #X_jd = yd con yd = 1
        for i in range(n_available_depots):
            xjd = []
            for j in range(n_customers):
                offset = j*n_available_depots + n_available_depots*n_customers
                #print(str(offset) + " vs " + str(len(distancias)) + " vs " + str(len(x)))
                xjd.append(x[offset])
            mdl.add_constraint(sum(xjd) == 1)

            
        #X_ij = 1
        for i in range(n_customers):
            aux = []
            for j in range(n_available_depots):
                aux.append(x[n_available_depots*i + j + len(distancias_d2c)])
            for j in range(n_customers-1):
                aux.append(x[(n_customers-1)*i + j + len(distancias_d2c) + len(distancias_c2d)])
            mdl.add_constraint(sum(aux) == 1)
        
        #X_ji = 1
        for i in range(n_customers):
            aux = []
            for j in range(n_available_depots):
                aux.append(x[j*n_available_depots + i])
            for j in range(n_customers):
                if(j == i):
                    continue
                aux.append(x[j*(n_customers-1) + i + len(distancias_d2c) + len(distancias_c2d)])
            mdl.add_constraint(sum(aux) == 1)

        
        # DL para F-MDRP
        u = mdl.continuous_var_list(n_customers, name="u")
        for i in range(n_customers):
            offset_ij = (n_customers-1)*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = (n_customers-1)*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(u[i] - u[j] + (n_customers-n_available_depots+1)*x[j + offset_ij] + (n_customers-n_available_depots-1)*x[i + offset_ji]  <= n_customers - n_available_depots)
        
        
        # NODE-DL para MDRP
        '''
        k = mdl.continuous_var_list(n_customers, name="k")
        
        #Constraint 20
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(k[i] + (n_available_depots - (j+1))*x[j*n_customers + i] <= n_available_depots)
                
        #Constraint 21
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(k[i] + (n_available_depots - (j+1))*x[i*n_available_depots + j + len(distancias_d2c)] <= n_available_depots)
        
        #Constraint 22
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(1 + ((j+1)-1)*x[j*n_customers + i] <= k[i])
                
        #Constraint 23
        for i in range(n_customers):
            for j in range(n_available_depots):
                mdl.add_constraint(1 + ((j+1) - 1)*x[i*n_available_depots + j + len(distancias_d2c)] <= k[i])
        
        #Constraint 24
        for i in range(n_customers):
            offset_ij = (n_customers-1)*i + len(distancias_d2c) + len(distancias_c2d)
            for j in range(n_customers):
                if i==j:
                    continue
                offset_ji = (n_customers-1)*j + len(distancias_d2c) + len(distancias_c2d)
                #print(str(j + offset_ij) + " y " + str(i + offset_ji) + " contra " + str(len(x)))
                mdl.add_constraint(k[i] - k[j] <= (n_available_depots - 1)*(1 - x[j + offset_ij] - x[i + offset_ji]))
        '''
        
        mdl.print_information()
        
        sol_start = time.time()
        solution = mdl.solve()
        sol_end = time.time()
        print("Tiempo en segundos: " + str(sol_end - sol_start))
        # Verificar si se encontró una solución
        if solution:
            # Extraer el valor óptimo de la función objetivo
            print(f"Valor óptimo de la función objetivo: {solution.objective_value}")
        else:
            print("No se encontró solución óptima.")
            
        print("Caminos usados: " + str(len(solution.get_value_list(x))))

        #solution.display()

Tiempo desde el inicio del programa: 0.0010030269622802734
Siguiente archivo
coordDas88.dat

Número de customers: 88
Número de depots disponibles: 8
Model: MDRP
 - number of variables: 9152
   - binary=9064, integer=0, continuous=88
 - number of constraints: 7848
   - linear=7848
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Tiempo en segundos: 0.22600102424621582
Valor óptimo de la función objetivo: 151.83304692876115
Caminos usados: 9064
Tiempo desde el inicio del programa: 1.017003059387207
Siguiente archivo
coordMin27.dat

Número de customers: 27
Número de depots disponibles: 5
Model: MDRP
 - number of variables: 999
   - binary=972, integer=0, continuous=27
 - number of constraints: 766
   - linear=766
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Tiempo en segundos: 0.024001598358154297
Valor óptimo de la función objetivo: 1802.5694339303
Caminos usados: 972
Tiempo desde el inicio del programa: 1.127002477645874
Siguiente archiv